Attempt using Bagging

In [44]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import itertools
from sklearn import cross_validation, preprocessing, tree
from sklearn.metrics import classification_report, confusion_matrix

np.random.seed(15)

#read in datasets
teams = pd.read_csv('Teams.csv')
seasons = pd.read_csv('Seasons.csv')
tourneyCompactResults = pd.read_csv('TourneyCompactResults.csv')
tourneyDetailedResults = pd.read_csv('TourneyDetailedResults.csv')
tourneySeeds = pd.read_csv('TourneySeeds.csv')
tourneySlots = pd.read_csv('TourneySlots.csv')
regularSeasonCompactResults = pd.read_csv('RegularSeasonCompactResults.csv')
regularSeasonDetailedResults = pd.read_csv('RegularSeasonDetailedResults.csv')
submission_df = pd.read_csv( 'sample_submission.csv')

#turn all datasets into DataFrames
teams_df = pd.DataFrame(teams)
seasons_df = pd.DataFrame(seasons)
tourneyCompact_df = pd.DataFrame(tourneyCompactResults)
tourneyDetailed_df = pd.DataFrame(tourneyDetailedResults)
tourneySeeds_df = pd.DataFrame(tourneySeeds)
tourneySlots_df = pd.DataFrame(tourneySlots)
regSeasonCompact_df = pd.DataFrame(regularSeasonCompactResults)
regSeasonDetailed_df = pd.DataFrame(regularSeasonDetailedResults)

In [45]:
#turn all datasets into DataFrames
teams_df = pd.DataFrame(teams)
seasons_df = pd.DataFrame(seasons)
tourneyCompact_df = pd.DataFrame(tourneyCompactResults)
tourneyDetailed_df = pd.DataFrame(tourneyDetailedResults)
tourneySeeds_df = pd.DataFrame(tourneySeeds)
tourneySlots_df = pd.DataFrame(tourneySlots)
regSeasonCompact_df = pd.DataFrame(regularSeasonCompactResults)
regSeasonDetailed_df = pd.DataFrame(regularSeasonDetailedResults)

In [46]:
# Transform the data, get the seed number for winning and losing team and remove unnecessary columns
tourney_merged_df = pd.merge(left=tourneyCompact_df, right=tourneySeeds_df, how='left',
                           left_on=['Season', 'Wteam'], right_on=['Season', 'Team'])
tourney_merged_df.rename(columns={'Seed' : 'Wseed', 'Team' : 'W_team'}, inplace=True)
tourney_merged_df = pd.merge(left=tourney_merged_df, right=tourneySeeds_df, how='left',
                           left_on=['Season', 'Lteam'], right_on=['Season', 'Team'])
tourney_merged_df.rename(columns={'Seed' : 'Lseed', 'Team' : 'L_team'}, inplace=True)

In [47]:
tourney_sanitized_df = tourney_merged_df[['Season', 'Wteam', 'Lteam', 'Wseed', 'Lseed']].copy()
tourney_sanitized_df['Wseed'] = tourney_sanitized_df['Wseed'].map(lambda x: int(x[1:3]))
tourney_sanitized_df['Lseed'] = tourney_sanitized_df['Lseed'].map(lambda x: int(x[1:3]))
tourney_sanitized_df['Seed_diff'] = tourney_sanitized_df['Wseed'] - tourney_sanitized_df['Lseed']


In [48]:
tourney_winning_df = pd.DataFrame()
tourney_winning_df[['Team1', 'Team2', 'Seed_diff']] = tourney_sanitized_df[['Wteam', 'Lteam', 'Seed_diff']].copy()
tourney_winning_df['result'] = 1
tourney_losing_df = pd.DataFrame()
tourney_losing_df[['Team1', 'Team2', 'Seed_diff']] = tourney_sanitized_df[['Lteam', 'Wteam', 'Seed_diff']].copy()
tourney_losing_df['Seed_diff'] = -tourney_losing_df['Seed_diff']
tourney_losing_df['result'] = 0
tourney_train_final_df = pd.concat((tourney_winning_df, tourney_losing_df))
team_dict = {t: i for i, t in enumerate(tourney_train_final_df.Team1.unique())}

In [49]:
X_train = tourney_train_final_df[['Team1', 'Team2', 'Seed_diff']].copy()
X_train['Team1'] = X_train['Team1'].apply(lambda x: team_dict[x])
X_train['Team2'] = X_train['Team2'].apply(lambda x: team_dict[x])
X_train['Seed_diff'] = X_train['Seed_diff'].values.reshape(-1,1)
Y_train = tourney_train_final_df.result.values

In [50]:
X_train, X_test, y_train, y_test = cross_validation.train_test_split(
    X_train, Y_train, test_size=0.2, random_state=5)

In [51]:
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.grid_search import GridSearchCV

n_estimators_list = [5,10,50,100,200,300,400,1000]

bagging = BaggingClassifier(KNeighborsClassifier(), max_samples=0.5, max_features=0.5)
grid = GridSearchCV(estimator=bagging, param_grid=dict(n_estimators=n_estimators_list))
grid.fit(X_train, y_train)

print(grid)
# summarize the results of the grid search
print(grid.best_score_)
print(grid.best_estimator_.n_estimators)

GridSearchCV(cv=None, error_score='raise',
       estimator=BaggingClassifier(base_estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'),
         bootstrap=True, bootstrap_features=False, max_features=0.5,
         max_samples=0.5, n_estimators=10, n_jobs=1, oob_score=False,
         random_state=None, verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'n_estimators': [5, 10, 50, 100, 200, 300, 400, 1000]},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)
0.702743902439
400


In [53]:
# Score of best_estimator on the left out data
print("best score is {0}".format(grid.best_score_))

# Print the optimized parameters used in the model selected from grid search
print "Params: ", grid.best_params_

### Assign the best estimator to final Extra Tree classifier
bagging_clf = grid.best_estimator_

best score is 0.702743902439
Params:  {'n_estimators': 400}


In [23]:
test_et_df = pd.DataFrame()
test_et_df['Season'] = submission_df['id'].apply(lambda x: int(x.split("_")[0]))
test_et_df['Team1'] = submission_df['id'].apply(lambda x: int(x.split("_")[1]))
test_et_df['Team2'] = submission_df['id'].apply(lambda x: int(x.split("_")[2]))

In [24]:
test_et_merged_df = pd.merge(left=test_et_df, right=tourneySeeds_df, how='left',
                           left_on=['Season', 'Team1'], right_on=['Season', 'Team'])
test_et_merged_df.rename(columns={'Seed' : 'Seed1', 'Team' : 'team_1'}, inplace=True)
test_et_merged_df = pd.merge(left=test_et_merged_df, right=tourneySeeds_df, how='left',
                           left_on=['Season', 'Team2'], right_on=['Season', 'Team'])
test_et_merged_df.rename(columns={'Seed' : 'Seed2', 'Team' : 'team_2'}, inplace=True)
test_et_sanitized_df = test_et_merged_df[['Season', 'Team1', 'Team2', 'Seed1', 'Seed2']].copy()
test_et_sanitized_df['Seed1'] = test_et_sanitized_df['Seed1'].map(lambda x: int(x[1:3]))
test_et_sanitized_df['Seed2'] = test_et_sanitized_df['Seed2'].map(lambda x: int(x[1:3]))
test_et_sanitized_df['Seed_diff'] = test_et_sanitized_df['Seed1'] - test_et_sanitized_df['Seed2']

In [25]:
X_et_test = test_et_sanitized_df[['Team1', 'Team2', 'Seed_diff']].copy()
X_et_test['Team1'] = X_et_test['Team1'].apply(lambda x: team_dict[x])
X_et_test['Team2'] = X_et_test['Team2'].apply(lambda x: team_dict[x])
X_et_test['Seed_diff'] = X_et_test['Seed_diff'].values.reshape(-1,1)

In [54]:
test_et_pred_prob = bagging_clf.predict_proba(X_et_test)[:,1]

In [55]:
final_et_output = pd.DataFrame()
final_et_output['id'] = submission_df['id']
final_et_output['pred'] = test_et_pred_prob
final_et_output.to_csv('./output/bagging.csv', index=False)